In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 


## LOADING DATA

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
df.head()

## PREPROCESSING

In [ ]:
#calculating CPM
def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),
                                              x['measurable_impressions'])*1000 , axis=1)

In [ ]:
df.drop(['integration_type_id' , 'revenue_share_percent'], axis = 1, inplace=True)

In [ ]:
df = df[df['CPM'].between(0, df['CPM'].quantile(.95))]

### working with ids

In [ ]:
for col in list(df.columns):
  if 'id' in col:
    tmp = pd.get_dummies(df[col], prefix=col, drop_first=True)
    df.drop([col], axis = 1, inplace=True)
    df = pd.concat([df, tmp], axis=1)

In [ ]:
df.head()

### Train test split

In [ ]:
test = df[pd.to_datetime(df.date) > pd.Timestamp(2019,6,21)]

In [ ]:
df = df[pd.to_datetime(df.date) <= pd.Timestamp(2019,6,21)]

In [ ]:
train = df[pd.to_datetime(df.date) <= pd.Timestamp(2019,6,19)]
val = df[pd.to_datetime(df.date) > pd.Timestamp(2019,6,19)]

In [ ]:
X_test = test[pd.to_datetime(test.date) > pd.Timestamp(2019,6,21)].drop(['CPM' , 'total_revenue', 'date'], axis = 1, inplace=False)

In [ ]:
X_test.shape

In [ ]:
X_train = train.drop(['CPM' , 'total_revenue', 'date'], axis = 1, inplace=False)
X_val = val.drop(['CPM' , 'total_revenue', 'date'], axis = 1, inplace=False)

In [ ]:
X_train.shape

In [ ]:
y_test = test['CPM']
y_train = train['CPM']
y_val = val['CPM']

Simple NN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()

model.add(Dense(X_train.shape[1],activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(optimizer='adam',loss='mse')


model.fit(x=X_train,y=y_train,
         validation_data=(X_val,y_val),
         batch_size=128,epochs=10)

In [ ]:
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred)

Interestingly, that if we drop colums 'order_id' , 'line_item_type_id', which we mentioned as unuseful by dataset author,  results are getting worse. Best MSE that I have achieved without these columns (with the same model) was approximately 4200. 